In [1]:
# import numpy as np
import pandas as pd
from pandas import DataFrame, Series

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

%matplotlib inline

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

/Users/alexguanga/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [25]:
# Importing the dataset
df = pd.read_csv("../archive/Churn_Modelling.csv")

In [26]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [27]:
# Using the apply function, for every column, we find the total amount of NULL/NA values
df.apply(lambda x: sum(x.isnull()))

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [28]:
# Looking at customers at a bank (They have measuring unusual churn rates: people are leaving the bank)
# 1: person stayed in the bank
# 0: person left the bank
# We have to predict models of potential people who might leave the bank!
# We will use indepedent variables to predict whether people will leave the bank

In [29]:
# Making Y-Variable
y = df['Exited']

# Including all but one column
Xs = df.iloc[:, 3:13]

In [30]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

labelencoder_X_geo = LabelEncoder()
Xs.loc[:, "Geography"] = labelencoder_X_geo.fit_transform(Xs.loc[:, "Geography"])
labelencoder_X_gender = LabelEncoder()
Xs.loc[:, "Gender"] = labelencoder_X_gender.fit_transform(Xs.loc[:, "Gender"])

onehotencoder = OneHotEncoder(categorical_features=[Xs.columns.get_loc("Geography")])
Xs = onehotencoder.fit_transform(Xs).toarray()

In [31]:
# Removing one of the dummy variables for the country
Xs = Xs[:, 1:]

In [32]:
# Splitting the data set into training and testing

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Xs, y, test_size=0.2, random_state=0)

In [33]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler

sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)

# We are only using the transform and not the fit transform because we already fit the model
X_test = sc_X.transform(X_test)

- Recall that we are using the units 6 as the average of the input and output layer (input=1, output=11, avg=6)

In [12]:
# Initilaizing the ANN model
classifier = Sequential()

# Adding the input layer
classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu', input_dim=11))

# Adding the second input layer (we are still using the recitifier function)
classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))

# Adding the final input layer (we are apply sigmoid function bc we need a prob of the outcomes happening)
classifier.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))


In [13]:
# Compiling the ANN: finding the best weight to use
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

- Epoch
    - An epoch describes the number of times the algorithm sees the entire data set. So, each time the algorithm has seen all samples in the dataset, an epoch has completed.

- Iteration
    - An iteration describes the number of times a batch of data passed through the algorithm. In the case of neural networks, that means the forward pass and backward pass. So, every time you pass a batch of data through the NN, you completed an iteration.

- Example: An example might make it clearer.
    - Say you have a dataset of 10 examples (or samples). You have a batch size of 2, and you've specified you want the algorithm to run for 3 epochs.
    - Therefore, in each epoch, you have 5 batches (10/2 = 5). Each batch gets passed through the algorithm, therefore you have 5 iterations per epoch. Since you've specified 3 epochs, you have a total of 15 iterations (5 * 3 = 15) for training.
    
- Batch size defines number of samples that going to be propagated through the network.
    - For instance, let's say you have 1050 training samples and you want to set up batch_size equal to 100. Algorithm takes first 100 samples (from 1st to 100th) from the training dataset and trains network. Next it takes second 100 samples (from 101st to 200th) and train network again. We can keep doing this procedure until we will propagate through the networks all samples. The problem usually happens with the last set of samples. In our example we've used 1050 which is not divisible by 100 without remainder. The simplest solution is just to get final 50 samples and train the network.

In [14]:
# Fit the model in the training set
# our batch size is 10 meaning we will be training 10 of the together (using the average) for 
# all the points 100 times!

classifier.fit(X_train, y_train, batch_size=10, epochs=100)

Epoch 1/100
8000/8000 [==============================] - 1s 186us/step - loss: 0.4912 - acc: 0.7952
Epoch 2/100
8000/8000 [==============================] - 1s 161us/step - loss: 0.4285 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 170us/step - loss: 0.4231 - acc: 0.7960
Epoch 4/100
8000/8000 [==============================] - 1s 163us/step - loss: 0.4193 - acc: 0.8171
Epoch 5/100
8000/8000 [==============================] - 2s 194us/step - loss: 0.4164 - acc: 0.8262
Epoch 6/100
8000/8000 [==============================] - 2s 209us/step - loss: 0.4145 - acc: 0.8294
Epoch 7/100
8000/8000 [==============================] - 2s 212us/step - loss: 0.4132 - acc: 0.8307
Epoch 8/100
8000/8000 [==============================] - 3s 327us/step - loss: 0.4117 - acc: 0.8324
Epoch 9/100
8000/8000 [==============================] - 2s 246us/step - loss: 0.4106 - acc: 0.8326
Epoch 10/100
8000/8000 [==============================] - 2s 254us/step - loss: 0.4093 - acc: 0.8340

In [20]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [21]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1479,  116],
       [ 174,  231]])

In [22]:
tn, fp, fn, tp = cm.ravel()

total = tn+fp+fn+tp
accuracy = (tn+tp)/total
precision = tp/(tp+fp)
recall = tp/(tp+fn)
f1_score = 2*precision*recall/(precision+recall)


print(
    "true pos: {0}\n"
    "false pos: {1}\n"
    "true neg: {2}\n"
    "false neg: {3}\n".format(tp, fp, tn, fn))

print("""
Out of {0} reviews, the model got {1} correct,
Accuacy is: {2:.2f}%
Precision is: {3:.2f}%
Recall is: {4:.2f}%
F1 Score is: {5:.2f}%""".format(total, tn+tp, accuracy, precision, recall, f1_score))

true pos: 231
false pos: 116
true neg: 1479
false neg: 174


Out of 2000 reviews, the model got 1710 correct,
Accuacy is: 0.85%
Precision is: 0.67%
Recall is: 0.57%
F1 Score is: 0.61%


In [18]:
"""
Predict for:
Geography = France
Credit score = 600
Gender = male
Age = 4
Tenure = 3
Balance = 60000
# of Product = 2
Has Credit Card = Yes
Is Active Member = Yes
Estimated Salary = 50000
"""

new_cust = np.array([[0.0, 0, 600, 1, 4, 3, 60000, 2, 1, 1, 50000]])
new_pred = classifier.predict(sc_X.transform(new_cust))

new_pred = classifier.predict(new_cust)
new_pred = (new_pred > 0.5)

In [19]:
new_pred

array([[False]])

- **K-fold Validation**

In [17]:
# Recall that we ran the entire process (until the scaling the data again in this step)
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

'''
Initilaizing the ANN model
Adding the input layer
Adding the second input layer (we are still using the recitifier function)
Adding the final input layer (we are apply sigmoid function bc we need a prob of the outcomes happening)
Adam is a type of SDG (stochastic gradient descent)
'''


def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu', input_dim=11))
    classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))
    classifier.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))
    classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn=build_classifier, batch_size=10, epochs=100)
accuracies = cross_val_score(estimator=classifier, X=X_train, y=y_train, cv=10, n_jobs=-1)


Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
7200/7200 [==============================] - 3s 483us/step - loss: 0.4826 - acc: 0.7967
Epoch 2/100
7200/7200 [==============================] - 3s 486us/step - loss: 0.4847 - acc: 0.7956
Epoch 2/100
7200/7200 [==============================] - 3s 430us/step - loss: 0.4271 - acc: 0.7975
Epoch 3/100
7200/7200 [==============================] - 4s 501us/step - loss: 0.4233 - acc: 0.7967
Epoch 4/100
7200/7200 [==============================] - 4s 501us/step - loss: 0.4210 - acc: 0.7975
Epoch 4/100
7200/7200 [==============================] - 3s 476us/step - loss: 0.4195 - acc: 0.8131
Epoch 5/100
7200/7200 [==============================] - 3s 471us/step - loss: 0.4168 - acc: 0.8161
Epoch 5/100
7200/7200 [==============================] - 3s 477us/step - loss: 0.4199 - acc: 0.8181
Epoch 5/100
7200/7200 [==============================] - 3s 476us/step - loss: 0.4139 - acc: 0.8267
Epoch 6/100
7200/7200 [==============================] - 3s 482u

7200/7200 [==============================] - 2s 343us/step - loss: 0.4031 - acc: 0.8342
Epoch 31/100
7200/7200 [==============================] - 2s 328us/step - loss: 0.4037 - acc: 0.8349
Epoch 31/100
7200/7200 [==============================] - 3s 354us/step - loss: 0.4039 - acc: 0.8342
Epoch 32/100
7200/7200 [==============================] - 3s 388us/step - loss: 0.4030 - acc: 0.8337
Epoch 32/100
7200/7200 [==============================] - 3s 368us/step - loss: 0.4035 - acc: 0.8332
Epoch 32/100
7200/7200 [==============================] - 2s 323us/step - loss: 0.3954 - acc: 0.8361
Epoch 33/100
7200/7200 [==============================] - 2s 335us/step - loss: 0.4038 - acc: 0.8349
Epoch 33/100
7200/7200 [==============================] - 2s 321us/step - loss: 0.4030 - acc: 0.8347
Epoch 34/100
7200/7200 [==============================] - 2s 318us/step - loss: 0.3952 - acc: 0.8364
Epoch 34/100
7200/7200 [==============================] - 2s 314us/step - loss: 0.4039 - acc: 0.8349
Epo

7200/7200 [==============================] - 3s 353us/step - loss: 0.4014 - acc: 0.8354
Epoch 61/100
7200/7200 [==============================] - 3s 371us/step - loss: 0.3928 - acc: 0.8358
Epoch 61/100
7200/7200 [==============================] - 3s 359us/step - loss: 0.4023 - acc: 0.8337
Epoch 62/100
7200/7200 [==============================] - 3s 378us/step - loss: 0.3927 - acc: 0.8350
Epoch 62/100
7200/7200 [==============================] - 3s 371us/step - loss: 0.4024 - acc: 0.8353
Epoch 63/100
7200/7200 [==============================] - 3s 392us/step - loss: 0.4007 - acc: 0.8361
Epoch 63/100
7200/7200 [==============================] - 3s 383us/step - loss: 0.4011 - acc: 0.8335
Epoch 63/100
7200/7200 [==============================] - 4s 557us/step - loss: 0.4016 - acc: 0.8337
Epoch 64/100
7200/7200 [==============================] - 4s 546us/step - loss: 0.3924 - acc: 0.8362
Epoch 64/100
7200/7200 [==============================] - 3s 467us/step - loss: 0.4023 - acc: 0.8346
Epo

7200/7200 [==============================] - 2s 336us/step - loss: 0.3922 - acc: 0.8357
Epoch 91/100
7200/7200 [==============================] - 2s 323us/step - loss: 0.4002 - acc: 0.8340
Epoch 92/100
7200/7200 [==============================] - 3s 365us/step - loss: 0.3975 - acc: 0.8353
Epoch 92/100
7200/7200 [==============================] - 3s 370us/step - loss: 0.3913 - acc: 0.8351
Epoch 92/100
7200/7200 [==============================] - 3s 356us/step - loss: 0.3973 - acc: 0.8372
Epoch 93/100
7200/7200 [==============================] - 3s 360us/step - loss: 0.4013 - acc: 0.8360
Epoch 94/100
7200/7200 [==============================] - 3s 379us/step - loss: 0.4002 - acc: 0.8362
Epoch 94/100
7200/7200 [==============================] - 3s 354us/step - loss: 0.3975 - acc: 0.8362
Epoch 94/100
7200/7200 [==============================] - 2s 342us/step - loss: 0.3916 - acc: 0.8358
Epoch 94/100
7200/7200 [==============================] - 3s 356us/step - loss: 0.4014 - acc: 0.8344
Epo

7200/7200 [==============================] - 3s 462us/step - loss: 0.4038 - acc: 0.8351
Epoch 22/100
7200/7200 [==============================] - 3s 474us/step - loss: 0.3990 - acc: 0.8357
Epoch 21/100
7200/7200 [==============================] - 4s 549us/step - loss: 0.4036 - acc: 0.8350
Epoch 23/100
7200/7200 [==============================] - 4s 605us/step - loss: 0.4078 - acc: 0.8318
Epoch 24/100
7200/7200 [==============================] - 4s 563us/step - loss: 0.4038 - acc: 0.8349
Epoch 24/100
7200/7200 [==============================] - 5s 628us/step - loss: 0.4057 - acc: 0.8347
Epoch 24/100
7200/7200 [==============================] - 4s 572us/step - loss: 0.3974 - acc: 0.8358
Epoch 24/100
7200/7200 [==============================] - 4s 578us/step - loss: 0.4074 - acc: 0.8322
Epoch 26/100
7200/7200 [==============================] - 4s 566us/step - loss: 0.4029 - acc: 0.8367
Epoch 26/100
7200/7200 [==============================] - 4s 553us/step - loss: 0.3965 - acc: 0.8382
Epo

7200/7200 [==============================] - 3s 378us/step - loss: 0.3906 - acc: 0.8392
Epoch 51/100
7200/7200 [==============================] - 3s 387us/step - loss: 0.4000 - acc: 0.8357
Epoch 53/100
7200/7200 [==============================] - 3s 397us/step - loss: 0.3903 - acc: 0.8390
Epoch 52/100
7200/7200 [==============================] - 3s 378us/step - loss: 0.4040 - acc: 0.8350
Epoch 54/100
7200/7200 [==============================] - 3s 386us/step - loss: 0.3996 - acc: 0.8357
Epoch 54/100
7200/7200 [==============================] - 3s 384us/step - loss: 0.3892 - acc: 0.8393
Epoch 53/100
7200/7200 [==============================] - 3s 414us/step - loss: 0.3995 - acc: 0.8365
Epoch 55/100
7200/7200 [==============================] - 3s 453us/step - loss: 0.3879 - acc: 0.8408
Epoch 54/100
7200/7200 [==============================] - 4s 517us/step - loss: 0.4041 - acc: 0.8346
Epoch 56/100
7200/7200 [==============================] - 4s 513us/step - loss: 0.3996 - acc: 0.8371
Epo

7200/7200 [==============================] - 2s 318us/step - loss: 0.3989 - acc: 0.8367
Epoch 84/100
7200/7200 [==============================] - 2s 327us/step - loss: 0.3438 - acc: 0.8593
Epoch 84/100
7200/7200 [==============================] - 2s 328us/step - loss: 0.3995 - acc: 0.8358
Epoch 85/100
7200/7200 [==============================] - 2s 322us/step - loss: 0.3988 - acc: 0.8351
Epoch 86/100
7200/7200 [==============================] - 2s 327us/step - loss: 0.3439 - acc: 0.8574
Epoch 85/100
7200/7200 [==============================] - 2s 321us/step - loss: 0.3988 - acc: 0.8357
Epoch 87/100
Epoch 86/100
7200/7200 [==============================] - 2s 326us/step - loss: 0.4033 - acc: 0.8335
Epoch 88/100
7200/7200 [==============================] - 2s 326us/step - loss: 0.3994 - acc: 0.8379
Epoch 88/100
7200/7200 [==============================] - 2s 347us/step - loss: 0.3992 - acc: 0.8367
Epoch 87/100
7200/7200 [==============================] - 2s 328us/step - loss: 0.3417 - ac

7200/7200 [==============================] - 1s 180us/step - loss: 0.3908 - acc: 0.8385
Epoch 37/100
7200/7200 [==============================] - 1s 177us/step - loss: 0.3669 - acc: 0.8489
Epoch 37/100
7200/7200 [==============================] - 1s 169us/step - loss: 0.3909 - acc: 0.8383
Epoch 38/100
7200/7200 [==============================] - 1s 168us/step - loss: 0.3645 - acc: 0.8469
Epoch 38/100
7200/7200 [==============================] - 1s 168us/step - loss: 0.3910 - acc: 0.8362
Epoch 39/100
7200/7200 [==============================] - 1s 170us/step - loss: 0.3916 - acc: 0.8375
Epoch 40/100
7200/7200 [==============================] - 1s 173us/step - loss: 0.3619 - acc: 0.8496
Epoch 40/100
7200/7200 [==============================] - 1s 180us/step - loss: 0.3913 - acc: 0.8368
Epoch 41/100
7200/7200 [==============================] - 1s 176us/step - loss: 0.3583 - acc: 0.8519
Epoch 41/100
7200/7200 [==============================] - 1s 170us/step - loss: 0.3906 - acc: 0.8381
Epo

In [19]:
# The accuracies for the 10 fold validation is around 84%
np.mean(accuracies)

0.8431249954551457

In [20]:
# The variance for the 10 fold validation is around 84%
np.std(accuracies)

0.012031858271698012

- Improving the ANN
- Dropout regularization to reduce overfitting if needed
    - This works by randonomly disabling neurons which will prevent the overfit, we will not have as much information if we included everything
- Dropout can be applied on a level basis. They should be applied to all the layers when the model is to overfit
- The p parameter is the total percentage of all the nuerons should be disabled
- A good recommendation is with .1 (10%)

In [ ]:
# Adding the input layer
classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu', input_dim=11))
classifier.add(Dropout(p=0.1))

# Adding the second input layer (we are still using the recitifier function)
classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))
classifier.add(Dropout(p=0.1))

# Adding the final input layer (we are apply sigmoid function bc we need a prob of the outcomes happening)
classifier.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))
classifier.add(Dropout(p=0.1))

# Compiling the ANN: finding the best weight to use
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit the model in the training set
# our batch size is 10 meaning we will be training 10 of the together (using the average) for 
# all the points 100 times!

classifier.fit(X_train, y_train, batch_size=10, epochs=100)

- Will be implementing the GridSearch to tune the model

In [ ]:
# Recall that we ran the entire process (until the scaling the data again in this step)
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu', input_dim=11))
    classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))
    classifier.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))
    classifier.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn=build_classifier)


parameters = {
    'batch_size': [20, 25, 30],
    'epochs': [100, 500], 
    'optimizer': ['adam', 'rmsprop']
}

grid_search = GridSearchCV(estimator=classifier, 
                           param_grid=parameters,
                           scoring='accuracy',
                           cv=10)

grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracies = grid_search.best_score_

# BEST PARAMS WERE batch_size=30, epochs=500, optimizer='rmsprop'

Epoch 1/100
7200/7200 [==============================] - 1s 117us/step - loss: 0.5278 - acc: 0.7971
Epoch 2/100
7200/7200 [==============================] - 1s 81us/step - loss: 0.4345 - acc: 0.7971
Epoch 3/100
7200/7200 [==============================] - 1s 82us/step - loss: 0.4280 - acc: 0.7971
Epoch 4/100
7200/7200 [==============================] - 1s 80us/step - loss: 0.4248 - acc: 0.7971
Epoch 5/100
7200/7200 [==============================] - 1s 107us/step - loss: 0.4211 - acc: 0.7976
Epoch 6/100
7200/7200 [==============================] - 1s 106us/step - loss: 0.4183 - acc: 0.8185
Epoch 7/100
7200/7200 [==============================] - 1s 126us/step - loss: 0.4168 - acc: 0.8215
Epoch 8/100
7200/7200 [==============================] - ETA: 0s - loss: 0.4173 - acc: 0.824 - 1s 99us/step - loss: 0.4146 - acc: 0.8271
Epoch 9/100
7200/7200 [==============================] - 1s 100us/step - loss: 0.4141 - acc: 0.8286
Epoch 10/100
7200/7200 [==============================] - 1s 90us/

Epoch 64/100
7200/7200 [==============================] - 1s 147us/step - loss: 0.4004 - acc: 0.8369
Epoch 65/100
7200/7200 [==============================] - 1s 95us/step - loss: 0.3999 - acc: 0.8363
Epoch 66/100
7200/7200 [==============================] - 1s 110us/step - loss: 0.4002 - acc: 0.8371
Epoch 67/100
7200/7200 [==============================] - 1s 85us/step - loss: 0.4001 - acc: 0.8369
Epoch 68/100
7200/7200 [==============================] - 1s 75us/step - loss: 0.3998 - acc: 0.8375
Epoch 69/100
7200/7200 [==============================] - 1s 73us/step - loss: 0.4000 - acc: 0.8378
Epoch 70/100
7200/7200 [==============================] - 1s 83us/step - loss: 0.4000 - acc: 0.8389
Epoch 71/100
7200/7200 [==============================] - 1s 75us/step - loss: 0.3997 - acc: 0.8367
Epoch 72/100
7200/7200 [==============================] - 1s 75us/step - loss: 0.4004 - acc: 0.8367
Epoch 73/100
7200/7200 [==============================] - 1s 81us/step - loss: 0.3997 - acc: 0.838

7200/7200 [==============================] - 1s 70us/step - loss: 0.3936 - acc: 0.8361
Epoch 27/100
7200/7200 [==============================] - 1s 72us/step - loss: 0.3932 - acc: 0.8375
Epoch 28/100
7200/7200 [==============================] - 1s 70us/step - loss: 0.3934 - acc: 0.8385
Epoch 29/100
7200/7200 [==============================] - 1s 70us/step - loss: 0.3927 - acc: 0.8381
Epoch 30/100
7200/7200 [==============================] - 1s 69us/step - loss: 0.3929 - acc: 0.8375
Epoch 31/100
7200/7200 [==============================] - 1s 70us/step - loss: 0.3930 - acc: 0.8383
Epoch 32/100
7200/7200 [==============================] - 1s 70us/step - loss: 0.3926 - acc: 0.8378
Epoch 33/100
7200/7200 [==============================] - 1s 83us/step - loss: 0.3927 - acc: 0.8368
Epoch 34/100
7200/7200 [==============================] - 1s 90us/step - loss: 0.3923 - acc: 0.8382
Epoch 35/100
7200/7200 [==============================] - 1s 83us/step - loss: 0.3919 - acc: 0.8385
Epoch 36/100


7200/7200 [==============================] - 1s 73us/step - loss: 0.4036 - acc: 0.8331
Epoch 90/100
7200/7200 [==============================] - 1s 76us/step - loss: 0.4036 - acc: 0.8337
Epoch 91/100
7200/7200 [==============================] - 1s 71us/step - loss: 0.4035 - acc: 0.8335
Epoch 92/100
7200/7200 [==============================] - 1s 70us/step - loss: 0.4034 - acc: 0.8315
Epoch 93/100
7200/7200 [==============================] - 1s 71us/step - loss: 0.4037 - acc: 0.8317
Epoch 94/100
7200/7200 [==============================] - 1s 70us/step - loss: 0.4034 - acc: 0.8336
Epoch 95/100
7200/7200 [==============================] - 1s 79us/step - loss: 0.4034 - acc: 0.8339: 0s - loss: 0.3880 - a
Epoch 96/100
7200/7200 [==============================] - 1s 75us/step - loss: 0.4033 - acc: 0.8322
Epoch 97/100
7200/7200 [==============================] - 1s 72us/step - loss: 0.4033 - acc: 0.8326
Epoch 98/100
7200/7200 [==============================] - 1s 71us/step - loss: 0.4033 - ac

7200/7200 [==============================] - 1s 77us/step - loss: 0.4024 - acc: 0.8357
Epoch 52/100
7200/7200 [==============================] - 1s 77us/step - loss: 0.4021 - acc: 0.8357
Epoch 53/100
7200/7200 [==============================] - 1s 77us/step - loss: 0.4018 - acc: 0.8354
Epoch 54/100
7200/7200 [==============================] - 1s 151us/step - loss: 0.4021 - acc: 0.8356
Epoch 55/100
7200/7200 [==============================] - 1s 92us/step - loss: 0.4021 - acc: 0.8354
Epoch 56/100
7200/7200 [==============================] - 1s 86us/step - loss: 0.4015 - acc: 0.8344
Epoch 57/100
7200/7200 [==============================] - 1s 78us/step - loss: 0.4018 - acc: 0.8344
Epoch 58/100
7200/7200 [==============================] - 1s 177us/step - loss: 0.4016 - acc: 0.8365 0s - loss:
Epoch 59/100
7200/7200 [==============================] - 1s 94us/step - loss: 0.4015 - acc: 0.8358
Epoch 60/100
7200/7200 [==============================] - 1s 116us/step - loss: 0.4018 - acc: 0.8349

...

Epoch 82/100
7200/7200 [==============================] - 1s 111us/step - loss: 0.3419 - acc: 0.8603
Epoch 83/100
7200/7200 [==============================] - 1s 107us/step - loss: 0.3412 - acc: 0.8611
Epoch 84/100
7200/7200 [==============================] - 1s 108us/step - loss: 0.3412 - acc: 0.8596
Epoch 85/100
7200/7200 [==============================] - 1s 106us/step - loss: 0.3409 - acc: 0.8611
Epoch 86/100
7200/7200 [==============================] - 1s 106us/step - loss: 0.3405 - acc: 0.8622
Epoch 87/100
7200/7200 [==============================] - 1s 104us/step - loss: 0.3407 - acc: 0.8610
Epoch 88/100
7200/7200 [==============================] - 1s 105us/step - loss: 0.3415 - acc: 0.8610
Epoch 89/100
7200/7200 [==============================] - 1s 107us/step - loss: 0.3408 - acc: 0.8625
Epoch 90/100
7200/7200 [==============================] - 1s 106us/step - loss: 0.3401 - acc: 0.8618
Epoch 91/100
7200/7200 [==============================] - 1s 107us/step - loss: 0.3405 - ac

7200/7200 [==============================] - 1s 105us/step - loss: 0.4029 - acc: 0.8347
Epoch 45/100
7200/7200 [==============================] - 1s 103us/step - loss: 0.4029 - acc: 0.8365
Epoch 46/100
7200/7200 [==============================] - 1s 105us/step - loss: 0.4026 - acc: 0.8362
Epoch 47/100
7200/7200 [==============================] - 1s 105us/step - loss: 0.4024 - acc: 0.8365
Epoch 48/100
7200/7200 [==============================] - 1s 105us/step - loss: 0.4025 - acc: 0.8358
Epoch 49/100
7200/7200 [==============================] - 1s 106us/step - loss: 0.4023 - acc: 0.8356
Epoch 50/100
7200/7200 [==============================] - 1s 106us/step - loss: 0.4026 - acc: 0.8360
Epoch 51/100
7200/7200 [==============================] - 1s 105us/step - loss: 0.4023 - acc: 0.8356
Epoch 52/100
7200/7200 [==============================] - 1s 103us/step - loss: 0.4021 - acc: 0.8365
Epoch 53/100
7200/7200 [==============================] - 1s 106us/step - loss: 0.4022 - acc: 0.8360
Epo

7200/7200 [==============================] - 1s 119us/step - loss: 0.4255 - acc: 0.7957
Epoch 7/100
7200/7200 [==============================] - 1s 120us/step - loss: 0.4242 - acc: 0.7957
Epoch 8/100
7200/7200 [==============================] - 1s 121us/step - loss: 0.4223 - acc: 0.7957
Epoch 9/100
7200/7200 [==============================] - 1s 121us/step - loss: 0.4200 - acc: 0.7962
Epoch 10/100
7200/7200 [==============================] - 1s 117us/step - loss: 0.4179 - acc: 0.8154
Epoch 11/100
7200/7200 [==============================] - 1s 117us/step - loss: 0.4165 - acc: 0.8222
Epoch 12/100
7200/7200 [==============================] - 1s 123us/step - loss: 0.4151 - acc: 0.8253
Epoch 13/100
7200/7200 [==============================] - 2s 251us/step - loss: 0.4139 - acc: 0.8268
Epoch 14/100
7200/7200 [==============================] - 1s 146us/step - loss: 0.4128 - acc: 0.8297 1s - loss: 0.3
Epoch 15/100
7200/7200 [==============================] - 1s 100us/step - loss: 0.4119 - acc

7200/7200 [==============================] - 1s 103us/step - loss: 0.3931 - acc: 0.8375
Epoch 69/100
7200/7200 [==============================] - 1s 108us/step - loss: 0.3931 - acc: 0.8367
Epoch 70/100
7200/7200 [==============================] - 1s 103us/step - loss: 0.3929 - acc: 0.8381
Epoch 71/100
7200/7200 [==============================] - 1s 110us/step - loss: 0.3925 - acc: 0.8392
Epoch 72/100
7200/7200 [==============================] - 1s 105us/step - loss: 0.3924 - acc: 0.8407
Epoch 73/100
7200/7200 [==============================] - 1s 103us/step - loss: 0.3923 - acc: 0.8386
Epoch 74/100
7200/7200 [==============================] - 1s 102us/step - loss: 0.3922 - acc: 0.8389
Epoch 75/100
7200/7200 [==============================] - 1s 103us/step - loss: 0.3923 - acc: 0.8390
Epoch 76/100
7200/7200 [==============================] - 1s 102us/step - loss: 0.3923 - acc: 0.8404
Epoch 77/100
7200/7200 [==============================] - 1s 108us/step - loss: 0.3921 - acc: 0.8375
Epo

Epoch 130/500
7200/7200 [==============================] - 1s 121us/step - loss: 0.3965 - acc: 0.8354
Epoch 131/500
7200/7200 [==============================] - 1s 127us/step - loss: 0.3961 - acc: 0.8378
Epoch 132/500
7200/7200 [==============================] - 1s 114us/step - loss: 0.3959 - acc: 0.8369
Epoch 133/500
7200/7200 [==============================] - 1s 114us/step - loss: 0.3961 - acc: 0.8371
Epoch 134/500
7200/7200 [==============================] - 1s 114us/step - loss: 0.3960 - acc: 0.8372
Epoch 135/500
7200/7200 [==============================] - 1s 105us/step - loss: 0.3960 - acc: 0.8375
Epoch 136/500
7200/7200 [==============================] - 1s 106us/step - loss: 0.3959 - acc: 0.8357
Epoch 137/500
7200/7200 [==============================] - 22s 3ms/step - loss: 0.3961 - acc: 0.8367
Epoch 138/500
7200/7200 [==============================] - 1s 144us/step - loss: 0.3958 - acc: 0.8361
Epoch 139/500
7200/7200 [==============================] - 1s 122us/step - loss: 0.